 # Imports and Installs

In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

root_dir_path = os.getcwd()
root_dir_path = os.path.abspath(os.path.split(root_dir_path)[0])

In [2]:
from autogluon.common import space
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

# Model Training

In [14]:
model_save_path = rf'{root_dir_path}/models/'

train_df = pd.read_csv(rf'{root_dir_path}/data/train_val_csv.csv', delimiter= '|', encoding='utf-8', index_col=0)

train_data = TabularDataset(train_df)
label = "KPB AKTIVITETSKOD"

train_data.head(5)

,clean_code,account_code,counterpart_code,business_code,activity_code,supplier,KPB AKTIVITETSKOD
0,680000,30102,EP0000,1000,NaN,NaN,A_ÖVR_504_0
1,680000,49000,AÖKA00,1000,NaN,NaN,A_ÖVR_504_0
2,680000,50100,EP0000,1000,NaN,NaN,A_ÖVR_504_0
3,680000,50460,EP0000,1000,NaN,NaN,A_ÖVR_504_0
4,680000,55211,EP0000,1000,NaN,NaN,A_ÖVR_504_0


# Hyperparam Selection

In [15]:
hp_tune = True

nn_options = {
    'num_epochs': 10,
    'learning_rate': space.Real(1e-4, 1e-2, default=5e-4, log=True),
    'layers': space.Categorical([100],[1000],[200,100],[300,200,100]),
    'dropout_prob': space.Real(0.0, 0.5, default=0.1),
}

gbm_options = {
    'num_boost_round': 100,
    'num_leaves': space.Int(lower=26, upper=66, default=36),
}

hyperparameters = {
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,
                  }

time_limits = 2*60
num_trials = 5 
search_strategy = 'auto'

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
} 

In [16]:
column_predictor = TabularPredictor(label=label, path=model_save_path).fit(train_data,
    time_limit=time_limits,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.20
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          4
Memory Avail:       4.21 GB / 15.88 GB (26.5%)
Disk Space Avail:   63.31 GB / 236.39 GB (26.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 120s
AutoGluon will save mo

  0%|          | 0/5 [00:00<?, ?it/s]

	Ran out of time, early stopping on iteration 26. Best iteration is:
	[1]	valid_set's multi_error: 0.251153
	Stopping HPO to satisfy time limit...
Fitted model: LightGBM\T1 ...
	0.749	 = Validation score   (accuracy)
	49.48s	 = Training   runtime
	0.07s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch ... Tuning model for up to 53.75s of the 69.66s of remaining time.
Traceback (most recent call last):
  File "c:\Users\vicke\anaconda3\envs\Wizdo_2\lib\site-packages\ray\tune\search\hyperopt\hyperopt_search.py", line 254, in _lookup
    idx = categories.index(config_dict[key])
ValueError: [100] is not in list

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\vicke\anaconda3\envs\Wizdo_2\lib\site-packages\autogluon\core\trainer\abstract_trainer.py", line 2236, in _train_single_full
    hpo_models, hpo_results = model.hyperparameter_tune(
  File "c:\Users\vicke\anaconda3\envs\Wizdo_2\lib\site-packages\

In [19]:
column_predictor = TabularPredictor(label=label, path=model_save_path).fit(train_data, presets='high_quality')

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.9.20
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          4
Memory Avail:       4.47 GB / 15.88 GB (28.1%)
Disk Space Avail:   63.15 GB / 236.39 GB (26.7%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and

# Model Testing and Selection

In [20]:
val_data = TabularDataset(train_df)
val_data = val_data.sample(n=30000, random_state = 42)
val_data.drop(columns=['KPB AKTIVITETSKOD'])

val_predictor = TabularPredictor.load(model_save_path)

y_pred = val_predictor.predict(val_data)

In [21]:
val_predictor.leaderboard(val_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2_FULL,0.996267,NaN,accuracy,90.713551,NaN,352.074943,0.414694,NaN,38.190448,2,True,12
1,KNeighborsDist_BAG_L1_FULL,0.994333,NaN,accuracy,0.268339,0.466812,0.602914,0.268339,0.466812,0.602914,1,True,9
2,KNeighborsDist_BAG_L1,0.994333,0.793043,accuracy,0.280466,0.466812,0.602914,0.280466,0.466812,0.602914,1,True,2
3,WeightedEnsemble_L3_FULL,0.984833,NaN,accuracy,98.127441,NaN,414.719679,0.489663,NaN,43.414313,3,True,14
4,NeuralNetFastAI_BAG_L2_FULL,0.956933,NaN,accuracy,97.637779,NaN,371.305366,7.007469,NaN,56.866500,2,True,13
5,KNeighborsUnif_BAG_L1_FULL,0.803733,NaN,accuracy,0.331453,0.416510,0.554371,0.331453,0.416510,0.554371,1,True,8
6,KNeighborsUnif_BAG_L1,0.803733,0.672595,accuracy,0.561521,0.416510,0.554371,0.561521,0.416510,0.554371,1,True,1
7,LightGBMXT_BAG_L1_FULL,0.550167,NaN,accuracy,87.492453,NaN,73.204340,87.492453,NaN,73.204340,1,True,11
8,NeuralNetFastAI_BAG_L1_FULL,0.349200,NaN,accuracy,2.538066,NaN,240.077241,2.538066,NaN,240.077241,1,True,10
9,WeightedEnsemble_L3,NaN,0.832351,accuracy,NaN,416.426901,2261.715888,NaN,0.114512,43.414313,3,False,7
